# Guided exercise 2: Sections, guards and patterns

# Function's types

As we saw last week in Haskell everything has a type, which cannot be changed. Functions do always have a type, if we do not specify it, it will be automatically inferred.

The type of a fuction can be specified explicitly by:

* If the type is a basic type (like `Int`, `Integer`, `Float`, etc.): `nameOfFunction:: Type1 -> Type2 -> Type3 .. -> ResultType`
* If the type is a type class (like `Num`, `Integral`, `Ord`, `Eq`, `Fractional`, etc.):  `(Type1 a1, Type2 a2) => a1 -> a1 -> a1 -> a2`


In [2]:
addTwo x = x + 2
-- + can be used with Num and 2 is also Num, so the type is Num a => a -> a
-- this means: this function receives a Num and returns a Num
:t addTwo
addTwo 3
addTwo 3.0

addTwo :: forall {a}. Num a => a -> a

5

5.0

In [3]:
-- If we force a type inside the function, the type of the function changes
addTwo' x = x + 2 :: Int
:t addTwo'
addTwo' 3
-- This will fail as 3.0 is not an Int
addTwo' 3.0

addTwo' :: Int -> Int

5

: 

In [14]:
-- Forcing the function to work only with Int
addTwo'':: Int -> Int
addTwo'' x = x + 2

addTwo'' 3
-- This exceeds the range of Int,  like in many other languages a circular range is considered
addTwo'' 32482084032840328403280348
-- No problem with the first version as it also works with Integer
addTwo 32482084032840328403280348

-- To make it work with Integral
addTwo''' :: Integral a => a -> a
addTwo''' x = x + 2
:t addTwo'''

addTwo''' 32482084032840328403280348

5


<interactive>:1:10: warning: [GHC-97441] [-Woverflowed-literals] Literal 32482084032840328403280348 is out of the Int range -9223372036854775808..9223372036854775807
5603440348473385438

32482084032840328403280350

addTwo''' :: forall a. Integral a => a -> a

32482084032840328403280350

pure :: forall (f :: * -> *) a. Applicative f => a -> f a

You may notice that when the type class is inferred, as in `addTwo`, it shows `forall {a}. Num a => a -> a`, while when you explicitly specify it, as in `addTwo'''`, it shows `forall a. Integral a => a -> a` (no brackets surrounding the `a` in the `forall`). There is a slight difference among both approaches, which is out of the scope of this course, but briefly speaking, in the first case the compiler will select the most appropriate type when you invoke the function, while in the second case there are ways to specify the type and force the compiler to use it.

Some functions are polymorphic, meaning that they can receive inputs of any type or that their output can belong to any type, or both together.

In [15]:
-- Example of a function with polymorphic input
-- It converts anything to "Pepe", so the input can be of any type
toPepe x = "Pepe"
:t toPepe
toPepe 4
toPepe True

-- Example of a function with polymorphic input and output, our own id function
myId x = x
:t myId
myId "hello"

-- Examples of functions with polymorphic output are more strange, we will see one of them `pure` in the future

toPepe :: forall {p}. p -> String

"Pepe"

"Pepe"

myId :: forall {p}. p -> p

"hello"

# Functions as input

In Haskell functions can be the input of another function, given that the output type of one is compatible with the input type of the other. For example if one function returns an `Int` it can be the input of another that requires an `Int`

In [16]:
-- Function application is left-associative, this is equivalent to (myId toPepe) 5 -> toPepe 5 -> "Pepe"
myId toPepe 5
-- This will not work as it is equivalent to (toPepe myId) 5 -> "Pepe" 5 which makes no sense
toPepe myId 5

"Pepe"

: 

In [17]:
-- The way to solve it is
toPepe (myId 5)
-- This works! Whatever the input toPepe will return "Pepe"
toPepe myId

"Pepe"

"Pepe"

# Sections

An operator enclosed in a parenthesis with a value: `(operator value)`

In [18]:
-- Equivalent to 2 + 5
(+2) 5
-- They can be composed
(+2) ((*4) 2)
-- Or used in functions (most common use)
addTwo x = (+2) x
-- As the argument appears on both sides of the equality we can reduce it
-- as we would do in mathematics
addTwo' = (+2) 
addTwo' 3
-- Alternative
addTwo'' x = x + 2
addTwo'' 3
-- Another example
positive = (>0)
positive 5
positive 3.2
positive (-5)

Line 6: Eta reduce
Found:
addTwo x = (+ 2) x
Why not:
addTwo = (+ 2)

7

10

5

5

True

True

False

# Declaring functions with guards

Guards are a powerfull alternative to chained `if-then-else`, they allow to describe the output of a function depending on the specific values of the input.

In [8]:
-- A function to calculate the string mark given a numeric one 
mark :: Float -> String
mark m = 
    if m < 0 || m > 10 then "Wrong mark"
    else if m < 5 then "Fail"
    else if m < 7 then "Average"
    else if m < 9 then "Good"
    else if m < 10 then "Excellent"
    else "Honors"

mark (-1)
mark 5.1
mark 10.0
mark 9.99999
mark 10.1

Line 3: Use guards
Found:
mark m
  = if m < 0 || m > 10 then
        "Wrong mark"
    else
        if m < 5 then
            "Fail"
        else
            if m < 7 then
                "Average"
            else
                if m < 9 then "Good" else if m < 10 then "Excellent" else "Honors"
Why not:
mark m
  | m < 0 || m > 10 = "Wrong mark"
  | m < 5 = "Fail"
  | m < 7 = "Average"
  | m < 9 = "Good"
  | m < 10 = "Excellent"
  | otherwise = "Honors"

"Wrong mark"

"Average"

"Honors"

"Excellent"

"Wrong mark"

In [9]:
-- A function to calculate the string mark given a numeric one 
--(incomplete, not all patterns considered, notice there is no warning, it will be noticed in execution)
mark :: Float -> String
mark m
    | m < 0 || m > 10 = "Wrong mark"
    | m < 5 = "Fail"
    | m < 7 = "Average"
    | m < 9 = "Good"
    | m < 10 = "Excellent"

mark 3
mark 7
mark (-12)
mark 22
mark 10



"Fail"

"Good"

"Wrong mark"

"Wrong mark"

: 

In [10]:
-- A function to calculate the string mark given a numeric one 
mark :: Float -> String
mark m
    -- No need for all of them to be at the same column
    | m < 0 || m > 10 = "Wrong mark"
     | m < 5 = "Fail"
    | m < 7 = "Average"
      | m < 9 = "Good"
    | m < 10 = "Excellent"
    -- We can use m == 10 or rely on otherwise
    
 | otherwise = "Honors"
        
mark 3
mark 7
mark (-12)
mark 22
mark 10

"Fail"

"Good"

"Wrong mark"

"Wrong mark"

"Honors"

In [11]:
-- The show function: converting anything to a String
-- here we use it to concatenate numbers and string
mark :: Float -> String
mark m
    | m < 0 || m > 10 = "Wrong mark"
    | m < 5 = "Your mark is " ++ show m ++ ": Fail"
    | m < 7 = "Your mark is " ++ show m ++ ": Average"
    | m < 9 = "Your mark is " ++ show m ++ ": Good"
    | m < 10 = "Your mark is " ++ show m ++ ": Excellent"
    | otherwise = "Your mark is " ++ show m ++ ": Honors"

mark 3
mark 7
mark 22
mark (-12)

"Your mark is 3.0: Fail"

"Your mark is 7.0: Good"

"Wrong mark"

"Wrong mark"

In [12]:
-- Local definitions 
mark :: Float -> String
mark a 
        | a < 0 || a > 10 =  "Wrong mark"
        | a < 5 = out ++ "Fail"
        | a < 7 = out ++ "Average"
        | a < 9 = out ++"Good"
        | a < 10 = out ++"Excellent"
        | otherwise = out ++ "Honors"
        where out = "Your mark is " ++ show a ++ ": "

mark 3
mark 7
mark 22
mark (-12)

"Your mark is 3.0: Fail"

"Your mark is 7.0: Good"

"Wrong mark"

"Wrong mark"

In [21]:
-- Example of two local definitions
mark :: Float -> String
mark m
    | m < 0 = text2
    | m < 5 = text ++ ": Fail"
    | m < 7 = text ++ ": Average"
    | m < 9 = text ++ ": Good"
    | m < 10 = text ++ ": Excellent"
    | m == 10 = text ++ ": Honors"
    | otherwise = text2
    where 
     --- Indentation not required but easier to read
     text = "your mark is " ++ show m
     text2 = show m ++ " is an invalid grade"

mark 3
mark 7
mark 22
mark (-12)

"your mark is 3.0: Fail"

"your mark is 7.0: Good"

"22.0 is an invalid grade"

"-12.0 is an invalid grade"

# Declaring functions with patterns

Defining the result of the function for each input value. Not too useful with our current knowledge, we will see their utility in next labs.

In [14]:
-- Simple version of patterns, a value for each input, only works for Int
mark :: Int -> String
mark 0 = "Fail"
-- Instead of repeating the "fail" we use the value of
-- the previous pattern
mark 1 = mark 0
mark 2 = mark 0
mark 3 = mark 0
mark 4 = mark 0
mark 5 = "Average"
mark 6 = mark 5
-- Notice that mark 8 is not yet defined here, but it works
mark 7 = mark 8
mark 8 = "Good"
mark 9 = "Excellent"
mark 10 = "Honors"
-- The _ is like the otherwise for guards
mark _ = "Wrong mark"

mark 3
mark 7
mark 22
mark (-12)

"Fail"

"Good"

"Wrong mark"

"Wrong mark"

In [ ]:
-- Patterns, using an auxiliary function to group values, not too useful as it is easier with guards

groupMarks :: Float -> Int
groupMarks m
    | m < 0 || m > 10 = -1
    | m < 5 = 4
    | m < 7 = 5
    | m < 9 = 7
    | m < 10 = 9
    | m == 10 = 10

mark :: Int -> String
mark (-1) = "Wrong mark"
mark 4 =  "Fail"
mark 5 = "Average"
mark 7 = "Good"
mark 9 = "Excellent"
mark 10 = "Honors"

mark (groupMarks 3.2)
mark (groupMarks 7.9)
mark (groupMarks 22)
mark (groupMarks (-12))

"Fail"

"Good"

"Wrong mark"

"Wrong mark"